# Weekly Sales to Target Tracker Function

The objective of this function is to create tabular data used for creating a Weekly Sales to Target Tracker. This one function will return two files: a 'quarter to date' CSV and an 'annual to date' CSV.

Tableau Server Team/Rep Performance Sales Table is downloaded and fed into this function. This function also takes in the date of data retrieval, the quarter number, current number of billing days in the quarter, and current number of billing days in the year.

**Note:** Starting Q2, this dataframe will also include all NTX territory.

The function knows the finalized targets for each team, and uses this in conjunction with the imported sales data to create a dataframe; it also automatically performs calculations for all columns requested:
   * QTD/YTD Target
   * % to QTD/YTD Target 
   * % Growth PY
   * % to Quarter/Annual Target
    
The function will automatically display the tables it has created, as well as download those tables as CSV files to same path as this notebook. 

From there, the Power BI file is refreshed with the new data source.

## Data Acquisition

**Team/Rep Sales Performance**

Explore / Territory Leadership / West Zone Daily Sales (2022) Territory / Team/Rep Performance / 
Download Team_Rep Sales Table_crosstab

Following filters:
* QTD and YTD 
* Financial Sales
* Product Groups: CEMENT, EXTREMITIES, FOOT AND ANKLE, HIPS, KNEES, RESTORATIVE THERAPY, SPORTS MEDICINE, TRAUMA
* Direct Comparison and 2021 as PY
* Today's date selected; currently these updates are occurring on Friday mornings.

.txt file from Tableau crosstab download converted to .csv UTF-8 to be read by this function.


In [1]:
# Import necessary libraries

import pandas as pd
import numpy as np
from pathlib import Path

## Target Tracker Function

In [2]:
def target_tracker(date, q_number, q_current_billing_days, annual_current_billing_days):
    
    # Date is written 'mmddyy'
    # q_number written as string e.g.'1'
    
    # Establish quarter
    quarter_billing_days = {'1':63,'2':63,'3':64,'4':61}
    total_billing_days_quarter = quarter_billing_days[q_number]

    # Download final targets dataframe
    final_targets = pd.read_csv('final_targets_may_2022.csv', na_values=['0','0.0'])
    # Clean final targets dataframe
    final_targets.rename(columns={'GROUP':'group', 'TEAM': 'team_name', 'SALES MANAGER': 'sales_manager', 'PRODUCT GROUP': 'product_group', 
                              'Q1 TARGET': 'q1_target', 'Q2 TARGET': 'q2_target', 'Q3 TARGET': 'q3_target', 
                              'Q4 TARGET': 'q4_target', 'TOTAL TARGET': 'total_target', 'Q2-Q4 TARGET': 'q2_q4_target'},
                         inplace=True)
    final_targets = final_targets.replace('\$','', regex=True).replace(',','', regex=True).replace('-','0',regex=True)
    final_targets_annual = final_targets[['group','team_name', 'sales_manager', 'product_group', 'total_target']]

    # Download Tableau data
    quarterly_sales = pd.read_csv('tableau_q'+q_number+date+'.csv', na_values=['0','0.0','$0 ','$0','$0.00'])


    # Clean quarter dataframe
    quarterly_sales.rename(columns={'Team/Rep Name': 'team_name', 'Product Group': 'product_group',
                                    'Current Sales': 'current_sales', 'Previous Sales': 'sales_2021',
                                    'Growth %': 'percent_growth_py'},
                           inplace=True)
    quarterly_sales = quarterly_sales[['team_name', 'product_group', 'current_sales', 'sales_2021']]
    quarterly_sales = quarterly_sales.replace('\$','', regex=True).replace(',','', regex=True)
    quarterly_sales['product_group'] = quarterly_sales['product_group'].str.replace('&','AND',regex=True)
    quarterly_sales['product_group'] = quarterly_sales['product_group'].str.upper()
    quarterly_sales[['current_sales', 'sales_2021']] = quarterly_sales[['current_sales', 'sales_2021']].astype(float)
    quarterly_sales.fillna(0.01,inplace=True)
    combined_teams = ['TEAM KMR', 'BRYAN MASTERS', 'TEAM REDMAST']
    filtered_qtr = quarterly_sales[quarterly_sales['team_name'].isin(combined_teams)]
    filtered_qtr = filtered_qtr.groupby(by='product_group').sum()
    filtered_qtr.reset_index(inplace=True)
    filtered_qtr['team_name'] = 'KMR/REDMAST/BM*'    
    quarterly_sales = pd.concat([quarterly_sales, filtered_qtr])
    
    # Merge quarter dataframe with final targets
    qtr_merged = final_targets.merge(quarterly_sales, how='left', on=['team_name','product_group'])
    qtr_merged = qtr_merged[qtr_merged['product_group']!='TOTAL']

    # Quarter target/percentage calculations
    qtr_merged['qtd_target'] = ((abs(qtr_merged['q'+q_number+'_target']))/total_billing_days_quarter)*q_current_billing_days
    qtr_merged['percent_to_qtd_target'] = qtr_merged['current_sales']/qtr_merged['qtd_target']
    qtr_merged['percent_growth_py'] = (qtr_merged['current_sales']-qtr_merged['sales_2021'])/qtr_merged['sales_2021']
    qtr_merged['percent_to_qtr_target'] = qtr_merged['current_sales']/qtr_merged['q'+q_number+'_target']
    qtr_merged['team_name'] = qtr_merged['team_name'].str.replace('TEAM ','')

    # Split into Legacy and NTX teams
    large_qtr_merged = qtr_merged[qtr_merged['group']=='large']
    small_qtr_merged = qtr_merged[qtr_merged['group']=='small']
    large_qtr_merged.drop('group', axis=1, inplace=True)
    small_qtr_merged.drop('group', axis=1, inplace=True)
    
    qtr_merged.to_csv('OUTPUT_full_qtr.csv', index=False)

    #Annual Calculations: merge q1 with q2 for new annual
    q1 = pd.read_csv('q1_frozen.csv')
    q2 = pd.read_csv('tableau_q'+q_number+date+'.csv', na_values=['0','0.0','$0 ','$0','$0.00'])
    #Clean q1
    q1['product_group'] = q1['product_group'].str.upper()
    #Clean q2
    q2.rename(columns={'Team/Rep Name': 'team_name', 'Product Group': 'product_group',
                            'Current Sales': 'current_sales', 'Previous Sales': 'sales_2021',
                            'Growth %': 'percent_growth_py'},
                   inplace=True)
    q2 = q2[['team_name', 'product_group', 'current_sales', 'sales_2021']]
    q2 = q2.replace('\$','', regex=True).replace(',','', regex=True)
    q2['product_group'] = q2['product_group'].str.replace('&','AND',regex=True)
    q2['product_group'] = q2['product_group'].str.upper()
    q2[['current_sales', 'sales_2021']] = q2[['current_sales', 'sales_2021']].astype(float)
    #Merge q1+q2
    annual = q2.merge(q1, how='left', on=['team_name', 'product_group'])
    annual.fillna(0.01,inplace=True)
    #Recalculate annual = q1+q2 sales current and 2021
    annual['current_sales'] = annual['current_sales']+annual['q1_sales']
    annual['sales_2021'] = annual['sales_2021']+annual['q1_2021_sales']
    #Combine KMR,Redmast,BM
    filtered_annual = annual[annual['team_name'].isin(combined_teams)]
    filtered_annual = filtered_annual.groupby(by='product_group').sum()
    filtered_annual.reset_index(inplace=True)
    filtered_annual['team_name'] = 'KMR/REDMAST/BM*'
    annual = pd.concat([annual, filtered_annual])
    #Merge Targets
    annual_merged = final_targets_annual.merge(annual, how='left', on=['team_name','product_group'])
    annual_merged = annual_merged[annual_merged['product_group']!='TOTAL']
    annual_merged['team_name'] = annual_merged['team_name'].str.replace('TEAM ','')
    annual_merged.fillna(0.01, inplace=True)
    #Annual Calculations
    annual_merged['ytd_target'] = ((abs(annual_merged['total_target']))/251)*annual_current_billing_days
    annual_merged['percent_to_ytd_target'] = annual_merged['current_sales']/annual_merged['ytd_target']
    annual_merged['percent_growth_py'] = (annual_merged['current_sales']-annual_merged['sales_2021'])/annual_merged['sales_2021']
    annual_merged['percent_to_year_target'] = annual_merged['current_sales']/annual_merged['total_target']

    # Annual split into Legacy and NTX teams
    large_annual_merged = annual_merged[annual_merged['group']=='large']
    small_annual_merged = annual_merged[annual_merged['group']=='small']
    large_annual_merged.drop(['group', 'q1_sales', 'q1_2021_sales'], axis=1, inplace=True)
    small_annual_merged.drop(['group', 'q1_sales', 'q1_2021_sales'], axis=1, inplace=True)
    
    annual_merged.drop(['q1_sales', 'q1_2021_sales'], axis=1, inplace=True)
    annual_merged.to_csv('OUTPUT_full_annual.csv', index=False)
    
    # Download both new dataframes
    large_qtr_merged.to_csv('OUTPUT_large_quarter_sales_target_'+date+'.csv', index=False)
    small_qtr_merged.to_csv('OUTPUT_small_quarter_sales_target_'+date+'.csv', index=False)
    
    large_annual_merged.to_csv('OUTPUT_large_annual_sales_target_'+date+'.csv', index=False)
    small_annual_merged.to_csv('OUTPUT_small_annual_sales_target_'+date+'.csv', index=False)
    
    return 'Successful'

In [3]:
target_tracker('_06242022', '2', 58, 121)

C:\Users\dmusgrove\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


'Successful'

**Development Notes**


* str.replace updating soon; must include regex=True [documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.replace.html#pandas.Series.str.replace)